## Fine-tuning GPT on Dialogs Data (due date: July 4)

In this assignment we will fine-tune GPT model on dialogs data to make model predict next line in a dialog.

In [1]:
!pip install transformers==4.28.0 datasets evaluate bert_score --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.7 MB/s eta 0:00:00


We will use model [DialoGPT](https://huggingface.co/microsoft/DialoGPT-small).
Load the model and tokenizer from the pretrained model in transformers.

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_name="microsoft/DialoGPT-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### Preprocessing (20)

Load [Daily Dialog](https://huggingface.co/datasets/daily_dialog) dataset from trtansformers datasets.

In [5]:
from datasets import load_dataset


dataset = load_dataset("daily_dialog")

Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset daily_dialog downloaded and prepared to /root/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
})

The dataset contains dialogs split into lines, act and emotion labels corresponding to each line:
- act (communication intention):  \__dummy__ (0), inform (1), question (2), directive (3) and commissive (4)
- emotion: no emotion (0), anger (1), disgust (2), fear (3), happiness (4), sadness (5) and surprise (6).

We will train the model to generate answer based on previous line of the dialog. So we need to preprocess our data in pairs of context (previous line) and answer (next line).

For each dialog create several examples containing context (previous lines) and next line in the dialog. Let's take maximum length of context as 4 lines (3 previous lines and 1 current). In the end of context and answer add special token for the end of the string -- *tokenizer.eos_token* *(<|endoftext|>)*.

Here's the example of context samples created for a dialog.

**Dialog**:

```
  - Hi how are you?
  - Good, and you?
  - I'm on vacations
  - Great, have fun!
  ```




**New examples:**
```
  [context] Hi how are you? <|endoftext|>
  [answer] Good, and you? <|endoftext|>

  [context] Hi how are you? Good, and you? <|endoftext|>
  [answer] I'm on vacations <|endoftext|>

  [context] Hi how are you? Good, and you? I'm on vacations <|endoftext|>
  [answer] Great, have fun! <|endoftext|>

  [context] Good, and you? <|endoftext|>
  [answer] I'm on vacations <|endoftext|>

  [context] Good, and you? I'm on vacations <|endoftext|>
  [answer] Great, have fun! <|endoftext|>

  [context] I'm on vacations <|endoftext|>
  [answer] Great, have fun! <|endoftext|>

```

In [6]:
max_context_count = 4

In [7]:
def process_dialog(dialog):

  context_answer_dataset = []

  for index, line in enumerate(dialog):
    context = line.strip()
    for next_index in range(1, max_context_count):
      if next_index + index >= len(dialog):
        break
      answer = dialog [index + next_index].strip()
      context_with_eos = (context + ' ' + tokenizer.eos_token).strip()
      answer_witheos = (answer + ' ' + tokenizer.eos_token).strip()
      context_answer_dataset.append((context_with_eos, answer_witheos))
      context = context + ' ' + answer

  return context_answer_dataset



def process_dataset(dataset):
  context_answer_dataset = []
  for sample in dataset:
    dialog = sample['dialog']
    process_dialog(dialog)
    context_answer_dataset += process_dialog(dialog)

  context_answer_dataset = pd.DataFrame(context_answer_dataset, columns=['context', 'answer'])
  return context_answer_dataset

context_answer_dataset_train =  process_dataset(dataset['train'])
context_answer_dataset_validation =  process_dataset(dataset['validation'])
context_answer_dataset_test =  process_dataset(dataset['test'])
context_answer_dataset_train

,context,answer
0,"Say , Jim , how about going for a few beers af...",You know that is tempting but is really not go...
1,"Say , Jim , how about going for a few beers af...",What do you mean ? It will help us to relax . ...
2,"Say , Jim , how about going for a few beers af...",Do you really think so ? I don't . It will jus...
3,You know that is tempting but is really not go...,What do you mean ? It will help us to relax . ...
4,You know that is tempting but is really not go...,Do you really think so ? I don't . It will jus...
...,...,...
195280,"May I help you , sir ? I want a pair of locus ...",I need size 41 . <|endoftext|>
195281,I want a pair of locus . <|endoftext|>,"Take a look at the ones on display , please . ..."
195282,I want a pair of locus . Take a look at the on...,I need size 41 . <|endoftext|>
195283,"Take a look at the ones on display , please . ...",I need size 41 . <|endoftext|>


New dataset will contain much more examples that the original one.

Next we will preprocess our data using tokenizer.
Analyze length of lines in the dialogs to define maximum length for tokenization.

In [72]:
context_answer_dataset_train['tokenizer_length'] = context_answer_dataset_train.apply(lambda x: len(tokenizer.encode_plus(x['context'] + x['answer'])['input_ids']), axis=1)
context_answer_dataset_train

,context,answer,tokenizer_length
0,"Say , Jim , how about going for a few beers af...",You know that is tempting but is really not go...,32
1,"Say , Jim , how about going for a few beers af...",What do you mean ? It will help us to relax . ...,44
2,"Say , Jim , how about going for a few beers af...",Do you really think so ? I don't . It will jus...,68
3,You know that is tempting but is really not go...,What do you mean ? It will help us to relax . ...,30
4,You know that is tempting but is really not go...,Do you really think so ? I don't . It will jus...,54
...,...,...,...
195280,"May I help you , sir ? I want a pair of locus ...",I need size 41 . <|endoftext|>,35
195281,I want a pair of locus . <|endoftext|>,"Take a look at the ones on display , please . ...",23
195282,I want a pair of locus . Take a look at the on...,I need size 41 . <|endoftext|>,28
195283,"Take a look at the ones on display , please . ...",I need size 41 . <|endoftext|>,20


In [73]:
max_length = np.max(context_answer_dataset_train['tokenizer_length'])
context_answer_dataset_train['tokenizer_length'].describe()

count    195285.000000
mean         47.032281
std          26.517433
min           8.000000
25%          29.000000
50%          41.000000
75%          58.000000
max         453.000000
Name: tokenizer_length, dtype: float64

Create a function for preprocessing:
- join context and answer for each example
- tokenize joined examples using padding to maximum length (input_ids)
- define labels the same as input_ids.

To use padding we will need to set special token for padding in tokenizer.

In [22]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|endoftext|>'

In [74]:
def preprocessing_sample(sample):
  context = sample['context']
  answer = sample['answer']
  join_context_answer = context + answer
  sample_encode = tokenizer.encode_plus(join_context_answer,
                        max_length=max_length,
                        return_tensors='pt',
                        padding='max_length')

  sample_encode['labels'] = sample_encode['input_ids']
  return sample_encode

### Prediction with Pretrained Model (10)

Run the loaded pretrained model on 10 examples of test set.

In [75]:
def predict_samples(data):
  results = []
  contexts = []
  model.to(device)
  for index, row in data.iterrows():
    input_text = row['context']
    contexts.append(input_text)
    inputs = tokenizer.encode_plus(input_text, return_tensors="pt")
    output = model.generate(inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device), max_length=max_length)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    results.append(generated_text)
  return contexts, results

In [ ]:
samples_10 = context_answer_dataset_test.iloc[:10]
test_context, test_predictions = predict_samples(samples_10)

In [77]:

def print_predictions(contexts, answers, labels):
  assert(len(contexts) == len(answers) == len(labels))
  for context, answer, label in zip(contexts, answers, labels):

    print(f'Context: {context}\n')
    print(f'Answer: {answer}\n')
    print(f'Label: {label}\n\n')


print_predictions(test_context, test_predictions, list(samples_10['answer']))

Context: Hey man , you wanna buy some weed ? <|endoftext|>

Answer: Hey man, you wanna buy some weed? I'm not sure if you're being sarcastic or not.

Label: Some what ? <|endoftext|>


Context: Hey man , you wanna buy some weed ? Some what ? <|endoftext|>

Answer: Hey man, you wanna buy some weed? Some what? I'm not sure if you're being sarcastic or not, but I'm not sure if you're being sarcastic.

Label: Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>


Context: Hey man , you wanna buy some weed ? Some what ? Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>

Answer: Hey man, you wanna buy some weed? Some what? Weed! You know? Pot, Ganja, Mary Jane some chronic! I'm not sure if you're being sarcastic or not, but I'm not sure if you're being sarcastic.

Label: Oh , umm , no thanks . <|endoftext|>


Context: Some what ? <|endoftext|>

Answer: Some what? I think you're missing a few words there.

Label: Weed ! You know ? Pot , Ganja , Mary Jane

### Model Training (30)

Now we can fine-tune the model on the dialogs data.

Define training parameters with TrainingArguments from transformers and run training using Trainer. Notice that model training could use large memory so you might need to set smaller batch_size. You can train the model on some subset of train dataset to reduce training time.

In [78]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Create the dataset
train_dataset = Dataset.from_dict(context_answer_dataset_train.to_dict(orient='list'))
validation_dataset = Dataset.from_dict(context_answer_dataset_validation.to_dict(orient='list'))
test_dataset = Dataset.from_dict(context_answer_dataset_test.to_dict(orient='list'))


train_dataset = train_dataset.map(preprocessing_sample,  remove_columns=list(context_answer_dataset_train.columns))
validation_dataset = validation_dataset.map(preprocessing_sample,  remove_columns=list(context_answer_dataset_validation.columns))
test_dataset = test_dataset.map(preprocessing_sample,  remove_columns=list(context_answer_dataset_test.columns))


Map:   0%|          | 0/195285 [00:00<?, ? examples/s]

Map:   0%|          | 0/18247 [00:00<?, ? examples/s]

Map:   0%|          | 0/17262 [00:00<?, ? examples/s]

In [81]:
torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir='results',
    save_total_limit=2,
    load_best_model_at_end=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    evaluation_strategy='steps',
    logging_steps=200,
    eval_steps=200,
    save_steps=200,
    warmup_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.select(range(25000)),
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

trainer.train()


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
200,0.744600,0.322906
400,0.332100,0.313055
600,0.314100,0.307782
800,0.301300,0.305234
1000,0.293900,0.304053
1200,0.288500,0.303002
1400,0.284100,0.302975
1600,0.280700,0.303698
1800,0.273800,0.303709
2000,0.269900,0.303919


TrainOutput(global_step=4689, training_loss=0.28842131276455074, metrics={'train_runtime': 6201.6683, 'train_samples_per_second': 12.094, 'train_steps_per_second': 0.756, 'total_flos': 1.73386656e+16, 'train_loss': 0.28842131276455074, 'epoch': 3.0})

### Prediction with Fine-tuned Model (20)


Test the model on the same examples as you used with pretrained model before fine-tuning. Do you see the difference in results?

In [ ]:
test_predictions = predict_samples(context_answer_dataset_test.iloc[:10])

In [83]:
test_predictions

(['Hey man , you wanna buy some weed ? <|endoftext|>',
  'Hey man , you wanna buy some weed ? Some what ? <|endoftext|>',
  'Hey man , you wanna buy some weed ? Some what ? Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>',
  'Some what ? <|endoftext|>',
  'Some what ? Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>',
  'Some what ? Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! Oh , umm , no thanks . <|endoftext|>',
  'Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>',
  'Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! Oh , umm , no thanks . <|endoftext|>',
  'Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! Oh , umm , no thanks . I also have blow if you prefer to do a few lines . <|endoftext|>',
  'Oh , umm , no thanks . <|endoftext|>'],
 ["Hey man, you wanna buy some weed? I'm not sure. I'm not sure. ",
  "Hey man, you wanna buy some weed? Some what? I'm going to have to buy some cigarettes

**Experiments with Decoding**

Different decoding strategies for the output sequence also affect the results of text generation.

Check this [tutorial](https://huggingface.co/blog/how-to-generate) on implementation of greedy decoding, beam search or sampling with different parameters. Try to compare texts generated with different decoding methods for several examples. For sampling try to use different temperatue values.

If the results with fine-tuned model do not seem very good, you can experiment with decoding on pretrained model.

In [84]:
base_tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name)
base_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [103]:
def compare_sampling(sample, tempatures=[0.6, 0.7, 0.8, 0.9]):

  input_ids = torch.tensor(sample['input_ids']).to(device)
  attention_mask = torch.tensor(sample['attention_mask']).to(device)
  outputs= {}

  greedy_output = base_model.generate(input_ids, attention_mask=attention_mask, max_length=500)
  greedy_output= base_tokenizer.decode(greedy_output[0].to(device), skip_special_tokens=True)

  outputs['greedy_output'] = greedy_output
  outputs['beam_output'] = {}
  for tempature in tempatures:
    beam_output = base_model.generate(
      input_ids,
      attention_mask=attention_mask,
      max_length=500,
      num_beams=5,
      do_sample=True,
      top_p=0.9,
      temperature=tempature,
      early_stopping=True,
      num_return_sequences=5,
      no_repeat_ngram_size=2
  )
    beam_output = base_tokenizer.decode(beam_output[0], skip_special_tokens=True)
    outputs['beam_output'][tempature] = beam_output

  return outputs

In [104]:
samples_to_compare = test_dataset.select(range(5))
sample_outputs = []
for sample in samples_to_compare:
  output = compare_sampling(sample)
  sample_outputs.append(output)

In [105]:
for sample in sample_outputs:

  print("Output Greedy: ")
  print(sample['greedy_output'])
  for temp, text in sample['beam_output'].items():
    print(f"Beam Output for tempature {temp}: ")
    print(f'{text}\n\n')

Output Greedy: 
Hey man, you wanna buy some weed? Some what? I'm gonna need a lot more than some weed for this one.
Beam Output for tempature 0.6: 
Hey man, you wanna buy some weed? Some what? I think you're on to something


Beam Output for tempature 0.7: 
Hey man, you wanna buy some weed? Some what? I'm gonna need a lot more


Beam Output for tempature 0.8: 
Hey man, you wanna buy some weed? Some what? I think you're high


Beam Output for tempature 0.9: 
Hey man, you wanna buy some weed? Some what? I think you need some help


Output Greedy: 
Hey man, you wanna buy some weed? Some what? Weed! You know? Pot, Ganja, Mary Jane some chronic! I'm gonna need a lot more than a few weeds.
Beam Output for tempature 0.6: 
Hey man, you wanna buy some weed? Some what? Weed! You know? Pot, Ganja, Mary Jane some chronic! I'm going to go out on a limb here and say that you're a smoker.


Beam Output for tempature 0.7: 
Hey man, you wanna buy some weed? Some what? Weed! You know? Pot, Ganja, Mary J

We see small differences between different temature, we check the bean serch with tempature of 0.6,0.7,0.8,0.9. We can see also the output of greed search that is more different from the others. it's hard to say with output is the best.

### Evaluation (20)

Evaluate the model on test set.

Calculate [perplexity](https://huggingface.co/spaces/evaluate-metric/perplexity) for test set data.

In [ ]:
torch.cuda.empty_cache()

prediction_size = 10000
test_context, test_predictions = predict_samples(context_answer_dataset_test.iloc[:prediction_size])

In [92]:
print_predictions(test_context[0:10], test_predictions[0:10], list(samples_10['answer']))

Context: Hey man , you wanna buy some weed ? <|endoftext|>

Answer: Hey man, you wanna buy some weed? I'm not sure. 

Label: Some what ? <|endoftext|>


Context: Hey man , you wanna buy some weed ? Some what ? <|endoftext|>

Answer: Hey man, you wanna buy some weed? Some what? I ’ Ve got some. 

Label: Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>


Context: Hey man , you wanna buy some weed ? Some what ? Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>

Answer: Hey man, you wanna buy some weed? Some what? Weed! You know? Pot, Ganja, Mary Jane some chronic! What do you want to do? 

Label: Oh , umm , no thanks . <|endoftext|>


Context: Some what ? <|endoftext|>

Answer: Some what? I ’ m not sure. I ’ m not sure. 

Label: Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>


Context: Some what ? Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! <|endoftext|>

Answer: Some what? Weed! You know? Pot, Ganja, Mary Jan

In [93]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")

results = perplexity.compute(predictions=test_predictions, model_id=model_name, device='gpu')
results['mean_perplexity']

Using pad_token, but it is not set yet.


  0%|          | 0/625 [00:00<?, ?it/s]

2524.774571285343

Calculate [BERT-score](https://huggingface.co/spaces/evaluate-metric/bertscore) metrics for 100 examples from test set.

In [94]:
bertscore = load("bertscore")

references = list(context_answer_dataset_test['answer'].iloc[:prediction_size])
results = bertscore.compute(predictions=test_predictions, references=references, lang="en")
np.mean(results['precision'])

0.8391901399254799